In [1]:
# Source https://github.com/AlxDfy/OpenSea_API_DataScience/blob/main/data-mining-OpenSea.ipynb
# https://github.com/XamNalpak/opensea-procton-python/blob/main/.ipynb_checkpoints/test-checkpoint.ipynb
# https://github.com/adilmoujahid/moonbirds-data-analysis/blob/main/moonbirds-data-analysis.ipynb

In [2]:
from datetime import datetime, date, timedelta
import requests
import pandas as pd
import json

In [3]:
API_KEY = "NTdiM2E2OGYtYTZhMi00YzgzLWI3N2EtMjg4YzBjYThkM2Fj"

In [4]:
response_list = []
for i in range(0,60):
    if len(response_list) == 0:
        headers = {'Content-Type': 'application/json','Authorization':'Bearer '+API_KEY}
        data = '{"limit":200,"order_by":"sale_date","collection":"mutant-ape-yacht-club","order":"asc"}'
        response = requests.post('https://api.m3o.com/v1/nft/Assets', headers=headers, data=data)
        response_list.append(response)
    else:
        y = json.loads(response.text)
        try:
            next_page = y['previous']
            headers = {'Content-Type': 'application/json','Authorization':'Bearer '+API_KEY}
            data = '{"cursor":'+'"'+next_page+'"'+',"limit":200,"order_by":"sale_date","collection":"mutant-ape-yacht-club","order":"asc"}'
            response = requests.post('https://api.m3o.com/v1/nft/Assets', headers=headers, data=data)
            response_list.append(response)
        except KeyError:
            print("no previous site found...")
response_list

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [

In [5]:
print(len(response_list))
df_list = []
for i in response_list:
    
    response_df = pd.read_json(i.text)
    asset_df = pd.DataFrame(list(response_df['assets']))
    single_nft_data = asset_df[['token_id', 'sales','owner','presale', 'last_sale','traits']]

    def extract_nested_column_value(df, col, values_to_keep):
        result = []
        for j in values_to_keep:
            for i in df[col]:
                if i is None:
                    value = ""
                else:
                    value = i[j]
                result.append(value)
            se = pd.Series(result)
            df[col] = se.values
        return result, df

    owner_name = extract_nested_column_value(single_nft_data,'owner',['username'])[1]
    last_sale_date_id = extract_nested_column_value(owner_name,'last_sale',['transaction'])[1]
    last_sale_date = extract_nested_column_value(last_sale_date_id,'last_sale',['timestamp'])[1]

    df = last_sale_date
    def merge_dic(row):
        merged_dic = {}
        dic_list = row
        temp_dic = {}
        for k in dic_list[0].keys():
            if k == 'value' or k == 'trait_type':
                 temp_dic[k] = tuple( temp_dic[k] for  temp_dic in dic_list)

        for count, i in enumerate(temp_dic['trait_type']):
            merged_dic[i] =  temp_dic['value'][count]
        return merged_dic

    df['Q'] = df.apply(lambda row: merge_dic(row['traits']), axis=1)
    final_df = pd.concat([df.drop(['Q'], axis=1), df['Q'].apply(pd.Series)], axis=1)

    single_nft_data = final_df.drop(columns=['traits'])
    df_list.append(single_nft_data)

60


/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = se.values
/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Q'] = df.apply(lambda row: merge_dic(row['traits']), axis=1)
/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = se.values
/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Q'] = df.apply(lambda row: merge_dic(row['traits']), axis=1)
/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = se.values
/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Q'] = df.apply(lambda row: merge_dic(row['traits']), axis=1)
/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = se.values
/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Q'] = df.apply(lambda row: merge_dic(row['traits']), axis=1)
/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = se.values
/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Q'] = df.apply(lambda row: merge_dic(row['traits']), axis=1)
/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_58897/3149626512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [6]:
combined_df = pd.concat(df_list, axis=0, ignore_index=True)
print(combined_df['token_id'].is_unique)
combined_df

True


,token_id,sales,owner,presale,last_sale,Mouth,Eyes,Hat,Fur,Background,Clothes,Earring,Name
0,196,4,EDDYHVault,False,2022-05-07T20:47:21,M1 Bored Unshaven Pipe,M1 Cyborg,M1 Vietnam Era Helmet,M1 Brown,M1 Blue,NaN,NaN,NaN
1,192,4,,False,2022-01-06T15:10:48,M1 Bored Cigar,M1 Wide Eyed,M1 Beanie,M1 Red,M1 Purple,M1 Biker Vest,NaN,NaN
2,200,2,,False,2021-08-30T14:43:13,M2 Bored Unshaven Party Horn,M2 Angry,M2 Police Motorcycle Helmet,M2 Blue,M2 Orange,M2 Pimp Coat,NaN,NaN
3,198,2,,False,2021-08-29T02:59:24,M1 Bored Unshaven,M1 Bored,M1 Seaman's Hat,M1 Blue,M1 New Punk Blue,M1 Sailor Shirt,NaN,NaN
4,197,2,212vault,False,2021-11-15T15:21:25,M1 Bored Unshaven,M1 Robot,M1 Bayc Hat Black,M1 Tan,M1 Blue,M1 Tie Dye,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,23824,2,,False,2021-12-23T23:09:56,M1 Bored Unshaven Pipe,M1 Sleepy,M1 Fez,M1 Cream,M1 Army Green,M1 Tanktop,NaN,NaN
11996,21215,0,,False,,M2 Bored,M2 Sad,M2 Girl's Hair Short,M2 Death Bot,M2 New Punk Blue,M2 Toga,NaN,NaN
11997,12796,0,immi_io,False,,M1 Phoneme Oh,M1 Bored,M1 Bowler,M1 Cream,M1 Purple,NaN,NaN,NaN
11998,21956,0,friesframevault778,False,,M1 Bored Unshaven,M1 Eyepatch,M1 Seaman's Hat,M1 Cream,M1 Blue,M1 Admirals Coat,NaN,NaN


In [7]:
combined_df.to_csv('./data/single_nft_data.csv', index=False)